In [1]:
from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/My Drive/Datamining/TextMining/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp "/content/drive/My Drive/Datamining/TextMining/Notebooks/functions.py" .
import functions as f
import importlib
importlib.reload(f)
f.v()

2020-05-31 10:45


In [0]:
import pandas as pd

In [4]:
df = f.loadTaggedDatasets(path + "Dataset/")
df

,date,text,language_proba,month,is_repeated,is_spam,is_news
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False,False,False
1,2019-05-10,Bitcoin FUDsters RN https://t.co/Yxo3TCxQci,0.449887,5,False,False,True
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False,False,False
3,2019-05-11,$1000 left!,0.247924,5,False,False,False
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False,False,False
...,...,...,...,...,...,...,...
10329768,2019-11-23,Happy #FibonacciDay A while back I created a...,0.939878,11,False,True,True
10329769,2019-11-23,Bitcoin Suisse Certificates :) https://t.co/nd...,0.457228,11,False,False,True
10329770,2019-11-23,Register now for the early access of the Codug...,0.884063,11,False,True,False
10329771,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,0.925460,11,False,True,False


In [5]:
# Tiramos a la basura todo lo que tenga language_proba <= 0.5
df = df[df['language_proba'] > 0.5]
df

,date,text,language_proba,month,is_repeated,is_spam,is_news
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False,False,False
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False,False,False
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False,False,False
5,2019-05-13,#retweet &amp; tag 🎁🎁We are glad to announce i...,0.855190,5,False,True,True
6,2019-05-13,"The last time Bitcoin broke $8,000: Microsoft...",0.978768,5,True,False,True
...,...,...,...,...,...,...,...
10329767,2019-11-23,Daily profit for HODLING BTC since 2013 Data t...,0.924064,11,False,False,False
10329768,2019-11-23,Happy #FibonacciDay A while back I created a...,0.939878,11,False,True,True
10329770,2019-11-23,Register now for the early access of the Codug...,0.884063,11,False,True,False
10329771,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,0.925460,11,False,True,False


In [6]:
df_sum = df.groupby('date').size().reset_index(name = 'total_count')
df_sum

,date,total_count
0,2019-05-10,32093
1,2019-05-11,57226
2,2019-05-12,48499
3,2019-05-13,70764
4,2019-05-14,67155
...,...,...
193,2019-11-19,38783
194,2019-11-20,38278
195,2019-11-21,43713
196,2019-11-22,46193


In [7]:
df_group = df.groupby(by=['date', 'is_repeated']).size().reset_index(name = 'count')
df_group

,date,is_repeated,count
0,2019-05-10,False,30107
1,2019-05-10,True,1986
2,2019-05-11,False,54383
3,2019-05-11,True,2843
4,2019-05-12,False,45789
...,...,...,...
391,2019-11-21,True,2538
392,2019-11-22,False,43732
393,2019-11-22,True,2461
394,2019-11-23,False,19872


In [8]:
df_sum = df_sum.merge(
    df_group[df_group['is_repeated'] == True].drop(['is_repeated'], axis=1).rename(columns={'count': 'repeated_count'}),
    left_on='date', right_on='date', how='left')
df_sum = df_sum.merge(
    df_group[df_group['is_repeated'] == False].drop(['is_repeated'], axis=1).rename(columns={'count': 'not_repeated_count'}),
    left_on='date', right_on='date', how='left')
df_sum

,date,total_count,repeated_count,not_repeated_count
0,2019-05-10,32093,1986,30107
1,2019-05-11,57226,2843,54383
2,2019-05-12,48499,2710,45789
3,2019-05-13,70764,3810,66954
4,2019-05-14,67155,3808,63347
...,...,...,...,...
193,2019-11-19,38783,2533,36250
194,2019-11-20,38278,2421,35857
195,2019-11-21,43713,2538,41175
196,2019-11-22,46193,2461,43732


In [9]:
# Para spam solo tenemos en cuenta los no repetidos
df = df[df['is_repeated'] == False]
df

,date,text,language_proba,month,is_repeated,is_spam,is_news
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False,False,False
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False,False,False
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False,False,False
5,2019-05-13,#retweet &amp; tag 🎁🎁We are glad to announce i...,0.855190,5,False,True,True
9,2019-05-10,🚀🚀🚀 too the moon lol,0.726528,5,False,False,False
...,...,...,...,...,...,...,...
10329767,2019-11-23,Daily profit for HODLING BTC since 2013 Data t...,0.924064,11,False,False,False
10329768,2019-11-23,Happy #FibonacciDay A while back I created a...,0.939878,11,False,True,True
10329770,2019-11-23,Register now for the early access of the Codug...,0.884063,11,False,True,False
10329771,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,0.925460,11,False,True,False


In [10]:
df_group = df.groupby(by=['date', 'is_spam']).size().reset_index(name = 'count')
df_group

,date,is_spam,count
0,2019-05-10,False,20079
1,2019-05-10,True,10028
2,2019-05-11,False,37093
3,2019-05-11,True,17290
4,2019-05-12,False,29285
...,...,...,...
391,2019-11-21,True,15169
392,2019-11-22,False,29254
393,2019-11-22,True,14478
394,2019-11-23,False,12775


In [11]:
df_sum = df_sum.merge(
    df_group[df_group['is_spam'] == True].drop(['is_spam'], axis=1).rename(columns={'count': 'spam_count'}),
    left_on='date', right_on='date', how='left')
df_sum = df_sum.merge(
    df_group[df_group['is_spam'] == False].drop(['is_spam'], axis=1).rename(columns={'count': 'not_spam_count'}),
    left_on='date', right_on='date', how='left')
df_sum

,date,total_count,repeated_count,not_repeated_count,spam_count,not_spam_count
0,2019-05-10,32093,1986,30107,10028,20079
1,2019-05-11,57226,2843,54383,17290,37093
2,2019-05-12,48499,2710,45789,16504,29285
3,2019-05-13,70764,3810,66954,21559,45395
4,2019-05-14,67155,3808,63347,22211,41136
...,...,...,...,...,...,...
193,2019-11-19,38783,2533,36250,14480,21770
194,2019-11-20,38278,2421,35857,14567,21290
195,2019-11-21,43713,2538,41175,15169,26006
196,2019-11-22,46193,2461,43732,14478,29254


In [12]:
# Para noticias solo tenemos en cuenta los no spam
df = df[df['is_spam'] == False]
df

,date,text,language_proba,month,is_repeated,is_spam,is_news
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False,False,False
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False,False,False
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False,False,False
9,2019-05-10,🚀🚀🚀 too the moon lol,0.726528,5,False,False,False
10,2019-05-10,"$BTC all targets hit, more than 90% profit 💰",0.637279,5,False,False,False
...,...,...,...,...,...,...,...
10329762,2019-11-23,Bitcoin- just doing the SpaceX rocket factory,0.875814,11,False,False,False
10329763,2019-11-23,We are super happy to announce that TZC is add...,0.904834,11,False,False,False
10329764,2019-11-23,@EarlofTyrone @wenlembo @coindesk @La__Cuen @H...,0.972412,11,False,False,False
10329767,2019-11-23,Daily profit for HODLING BTC since 2013 Data t...,0.924064,11,False,False,False


In [13]:
df_group = df.groupby(by=['date', 'is_news']).size().reset_index(name = 'count')
df_group

,date,is_news,count
0,2019-05-10,False,15101
1,2019-05-10,True,4978
2,2019-05-11,False,27913
3,2019-05-11,True,9180
4,2019-05-12,False,20704
...,...,...,...
391,2019-11-21,True,6913
392,2019-11-22,False,20887
393,2019-11-22,True,8367
394,2019-11-23,False,8631


In [14]:
df_sum = df_sum.merge(
    df_group[df_group['is_news'] == True].drop(['is_news'], axis=1).rename(columns={'count': 'news_count'}),
    left_on='date', right_on='date', how='left')
df_sum = df_sum.merge(
    df_group[df_group['is_news'] == False].drop(['is_news'], axis=1).rename(columns={'count': 'not_news_count'}),
    left_on='date', right_on='date', how='left')
df_sum

,date,total_count,repeated_count,not_repeated_count,spam_count,not_spam_count,news_count,not_news_count
0,2019-05-10,32093,1986,30107,10028,20079,4978,15101
1,2019-05-11,57226,2843,54383,17290,37093,9180,27913
2,2019-05-12,48499,2710,45789,16504,29285,8581,20704
3,2019-05-13,70764,3810,66954,21559,45395,12009,33386
4,2019-05-14,67155,3808,63347,22211,41136,11169,29967
...,...,...,...,...,...,...,...,...
193,2019-11-19,38783,2533,36250,14480,21770,6292,15478
194,2019-11-20,38278,2421,35857,14567,21290,6352,14938
195,2019-11-21,43713,2538,41175,15169,26006,6913,19093
196,2019-11-22,46193,2461,43732,14478,29254,8367,20887


In [17]:
df_sum.to_csv(f'{path}Dataset/v1/tagged_summary.csv.gz',
                    index = False, compression='gzip', escapechar='\\', quotechar='"', encoding = "utf-8",
                    quoting=csv.QUOTE_NONNUMERIC, sep=";")

NameError: ignored